
#PEFT lab (finetuning meta-llama/Llama-3.2-1B-Instruct )

Fine-tuning

Q1: meta-llama/Llama-3.2-1B-Instruct를 사용

Q2: 데이터셋에서 2,800개의 샘플만을 사용하여 파인튜닝을 수행 (남은 200개 샘플은 검증(validation) 세트로 사용)

Q3: 파인튜닝된 LoRA 어댑터를 Hugging Face Hub에 업로드

Q4: Hugging Face Hub에서 어댑터 모델을 불러옴

Q5: 검증 세트의 출력과 생성된 출력에 대해 BLEU 점수를 측정

In [1]:
# 필요한 패키지 설치
!pip install transformers accelerate datasets peft trl bitsandbytes wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 9.6 MB/s eta 0:00:00


In [3]:
# 모듈 임포트
import os
from dataclasses import dataclass, field
from typing import Optional
import re

import torch
import sys

# import tyro
from accelerate import Accelerator
from datasets import load_dataset, Dataset
from peft import AutoPeftModelForCausalLM, LoraConfig
from tqdm import tqdm
from transformers import (
    HfArgumentParser,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    TextStreamer,
    logging as hf_logging,
)
import logging
from trl import SFTTrainer, SFTConfig

from trl.trainer import ConstantLengthDataset

## Q1: base model을  `meta-llama/Llama-3.2-1B-Instruct`로 설정

`meta-llama/Llama-3.2-1B-Instruct` 모델을 사용하도록 설정했습니다. 이를 위해 `base_model_id` 값을 수정했으며, 모델 초기화 및 학습 과정에서 필요한 코드도 해당 모델과 호환되도록 조정했습니다.



In [4]:
# 모델 및 데이터셋 설정
base_model_id = "meta-llama/Llama-3.2-1B-Instruct"  # 모델 이름 변경
device_map = "cuda"
torch_dtype = torch.bfloat16
output_dir = "./llama-order-analysis"  # 결과 저장 경로
dataset_name = "./llm-modeling-lab.jsonl"  # 데이터셋 경로
seq_length = 512

In [5]:
# 데이터셋 로드
full_dataset = Dataset.from_json(path_or_paths=dataset_name)

Generating train split: 0 examples [00:00, ? examples/s]

In [6]:
# Tokenizer 설정
tokenizer = AutoTokenizer.from_pretrained(base_model_id)
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [7]:
# 양자화 설정
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

In [8]:
# 모델 초기화(로드)
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=bnb_config,
    device_map="auto",
)
base_model.config.use_cache = False

if getattr(tokenizer, "pad_token", None) is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "right"  # Fix weird overflow issue with fp16 training
if base_model.config.pad_token_id != tokenizer.pad_token_id:
    base_model.config.pad_token_id = tokenizer.pad_token_id

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [9]:
# 데이터 처리 함수
def chars_token_ratio(dataset, tokenizer, prepare_sample_text, nb_examples=400):
    """
    Estimate the average number of characters per token in the dataset.
    """
    total_characters, total_tokens = 0, 0
    for _, example in tqdm(zip(range(nb_examples), iter(dataset)), total=nb_examples):
        text = prepare_sample_text(example)
        total_characters += len(text)
        if tokenizer.is_fast:
            total_tokens += len(tokenizer(text).tokens())
        else:
            total_tokens += len(tokenizer.tokenize(text))
    return total_characters / total_tokens

def function_prepare_sample_text(tokenizer, for_train=True):
    """A Closure"""
    def _prepare_sample_text(example):
        system_prompt = "이 모델은 주문 문장을 분석하는 역할을 합니다."
        user_prompt = "너는 사용자가 입력한 주문 문장을 분석하는 에이전트이다. 주문으로부터 이를 구성하는 음식명, 옵션명, 수량을 차례대로 추출해야 한다.\n### 주문 문장: "
        messages = [
            {"role": "system", "content": f"{system_prompt}"},
            {"role": "user", "content": f"{user_prompt}{example['input']}"},
        ]
        if for_train:
            messages.append({"role": "assistant", "content": f"{example['output']}"})
        text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False if for_train else True)
        return text
    return _prepare_sample_text

def create_datasets(tokenizer, dataset, seq_length):
    prepare_sample_text = function_prepare_sample_text(tokenizer)
    chars_per_token = chars_token_ratio(dataset, tokenizer, prepare_sample_text)
    print(f"The character to token ratio of the dataset is: {chars_per_token:.2f}")
    cl_dataset = ConstantLengthDataset(
        tokenizer,
        dataset,
        formatting_func=prepare_sample_text,
        infinite=True,
        seq_length=seq_length,
        chars_per_token=chars_per_token,
    )
    return cl_dataset


## Q2: 학습에는 2,800개, 검증에는 200개의 샘플 사용

데이터셋에서 학습용으로 2,800개, 검증용으로 200개의 샘플을 선택하기 위해 `.select()` 메서드를 활용했습니다. 이를 통해 모델 학습 및 검증이 명확히 구분된 데이터셋에서 이루어질 수 있도록 했습니다.


In [10]:
train_size = 2800
val_size = 200
train_dataset = full_dataset.select(range(train_size))
val_dataset = full_dataset.select(range(train_size, train_size + val_size))

# 데이터 전처리
ds_train = create_datasets(tokenizer, train_dataset, seq_length)
ds_val = create_datasets(tokenizer, val_dataset, seq_length)


100%|██████████| 400/400 [00:00<00:00, 1371.25it/s]


The character to token ratio of the dataset is: 2.57


 50%|█████     | 200/400 [00:00<00:00, 1432.93it/s]

The character to token ratio of the dataset is: 2.58


In [11]:
# 학습 데이터셋의 이터레이터 생성
it_train = iter(ds_train)

# 검증 데이터셋의 이터레이터 생성
it_validation = iter(ds_val)

In [12]:
# 학습 데이터셋에서 샘플 디코딩
train_sample_decoded = tokenizer.decode(next(it_train)['input_ids'])
print("Decoded text from training dataset:")
print(train_sample_decoded)

# 검증 데이터셋에서 샘플 디코딩
validation_sample_decoded = tokenizer.decode(next(it_validation)['input_ids'])
print("Decoded text from validation dataset:")
print(validation_sample_decoded)

/usr/local/lib/python3.10/dist-packages/trl/trainer/utils.py:648: UserWarning: The dataset reached end and the iterator is reset to the start.
  warnings.warn("The dataset reached end and the iterator is reset to the start.")


Decoded text from training dataset:
고,옵션:아이스,수량:한잔<|eot_id|><|eot_id|><|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 07 Dec 2024

이 모델은 주문 문장을 분석하는 역할을 합니다.<|eot_id|><|start_header_id|>user<|end_header_id|>

너는 사용자가 입력한 주문 문장을 분석하는 에이전트이다. 주문으로부터 이를 구성하는 음식명, 옵션명, 수량을 차례대로 추출해야 한다.
### 주문 문장: 피자가 먹고 싶은데, 크레이지핫치킨 L사이즈 한판, 포테이토 피자 M사이즈 한 판 주세요.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

- 분석 결과 0: 음식명:크레이지핫치킨,옵션:L,수량:한 판
- 분석 결과 1: 음식명:포테이토 피자,옵션:M,수량:한 판<|eot_id|><|eot_id|><|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 07 Dec 2024

이 모델은 주문 문장을 분석하는 역할을 합니다.<|eot_id|><|start_header_id|>user<|end_header_id|>

너는 사용자가 입력한 주문 문장을 분석하는 에이전트이다. 주문으로부터 이를 구성하는 음식명, 옵션명, 수량을 차례대로 추출해야 한다.
### 주문 문장: 한촌불고기 한 판이랑 참깨오레오베이글 하나 주세요.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

- 분석 결과 0: 음식명:한촌불고기,수량:한 판
- 분석 결과 1: 

In [13]:
lora_config = LoraConfig(
            r=8,
            lora_alpha=16,
            lora_dropout=0.05,
            target_modules=[
                "q_proj",
                "k_proj",
                "v_proj",
                "o_proj",
                "down_proj",
                "up_proj",
                "gate_proj",
            ],
            bias="none",
            task_type="CAUSAL_LM",
        )

In [14]:
peft_config = lora_config

In [15]:
from google.colab import userdata
import wandb

wandb_api_key = userdata.get('WANDB_API_KEY')
if wandb_api_key:
    wandb.login(key=wandb_api_key)
    print("Successfully logged in to Weights & Biases")
else:
    print("WANDB_API_KEY not found in Colab secrets")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Successfully logged in to Weights & Biases


In [16]:
# SFT 트레이너 설정
sft_config = SFTConfig(
    output_dir=output_dir,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=1,
    gradient_checkpointing=False,
    learning_rate=1e-4,
    warmup_ratio=0.1,
    max_grad_norm=0.3,
    weight_decay=0.05,
    num_train_epochs=1,
    logging_steps=20,
    eval_strategy="no",
    save_strategy="steps",
    save_steps=50,
    save_total_limit=2,
    max_seq_length=seq_length,
    report_to="wandb",
    run_name="meta-llama/Llama-3.2-1B-Instruct-tuning"
)

In [17]:
trainer = SFTTrainer(
    model=base_model,
    train_dataset=ds_train,
    eval_dataset=ds_val,
    peft_config=peft_config,
    tokenizer=tokenizer,
    args=sft_config,
)

In [18]:
# 모델 트레이닝 완료
trainer.train()

wandb: Currently logged in as: hjjummy (hjjummy-sungshin-women-s-university). Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.10/dist-packages/trl/trainer/utils.py:648: UserWarning: The dataset reached end and the iterator is reset to the start.
  warnings.warn("The dataset reached end and the iterator is reset to the start.")


Step,Training Loss
20,2.686300
40,2.285700
60,1.746200
80,1.246200
100,0.969200
120,0.885500
140,0.832800
160,0.793400
180,0.759400
200,0.684900


/usr/local/lib/python3.10/dist-packages/trl/trainer/utils.py:648: UserWarning: The dataset reached end and the iterator is reset to the start.
  warnings.warn("The dataset reached end and the iterator is reset to the start.")
/usr/local/lib/python3.10/dist-packages/trl/trainer/utils.py:648: UserWarning: The dataset reached end and the iterator is reset to the start.
  warnings.warn("The dataset reached end and the iterator is reset to the start.")


TrainOutput(global_step=1400, training_loss=0.6118387474332537, metrics={'train_runtime': 2472.8749, 'train_samples_per_second': 1.132, 'train_steps_per_second': 0.566, 'total_flos': 8419093040332800.0, 'train_loss': 0.6118387474332537, 'epoch': 1.0})

In [19]:
def wrapper_generate(tokenizer, model, input_prompt, do_stream=False):
    def get_text_after_prompt(text):
        # pattern = r'<start_of_turn>model\n(.*?)<end_of_turn>'
        # match = re.search(pattern, text, re.DOTALL)

        # if match:
        #     extracted_text = match.group(1).strip()
        #     return extracted_text
        # else:
        #     return "매칭되는 텍스트가 없습니다."
        #--------------------------------------------------------------
        # 정규식 패턴: "- 분석 결과"로 시작하는 텍스트 추출
        pattern = r"- 분석 결과 \d+:.*?(?=<|eot_id|>)"  # <|eot_id|> 이전까지 추출
        matches = re.findall(pattern, text, re.DOTALL)  # Dotall 옵션으로 다중 라인 처리
        return "\n".join(matches) if matches else "매칭되는 텍스트가 없습니다."

    data = tokenizer(input_prompt, return_tensors="pt")
    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
    input_ids = data.input_ids[..., :-1]
    with torch.no_grad():
        pred = model.generate(
            input_ids=input_ids.cuda(),
            streamer=streamer if do_stream else None,
            use_cache=True,
            max_new_tokens=128, #float("inf"),
            do_sample=False,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    decoded_text = tokenizer.batch_decode(pred, skip_special_tokens=False)

    # 결과에 대해 특별 처리
    return get_text_after_prompt(decoded_text[0])

In [20]:
from google.colab import userdata
from huggingface_hub import login

# Colab 환경 변수에서 Hugging Face 토큰 가져오기
hf_token = userdata.get("HF_TOKEN")
if hf_token:
    login(token=hf_token)
else:
    print("HF_TOKEN 환경 변수가 설정되지 않았습니다.")


## Q3: 학습된 LoRA 어댑터를 Hugging Face Hub에 업로드

`push_to_hub` 메서드를 사용하여 학습된 LoRA 어댑터를 Hugging Face Hub에 업로드했습니다. 모델의 ID는 `hjjummy/meta-llama-1b-lora-adapter_3`입니다.


In [21]:
# LoRA 어댑터 huggingface에 업로드 (Q3)
adapter_model_id = "hjjummy/meta-llama-1b-lora-adapter_3"

# 학습된 모델 업로드
trainer.model.push_to_hub(
    adapter_model_id,
    use_temp_dir=True,  # 임시 디렉토리를 사용해 모델 업로드
)
print(f"LoRA 어댑터 모델이 Hugging Face Hub에 업로드되었습니다: {adapter_model_id}")

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/22.6M [00:00<?, ?B/s]

LoRA 어댑터 모델이 Hugging Face Hub에 업로드되었습니다: hjjummy/meta-llama-1b-lora-adapter_2


## Q4: Hugging Face Hub에서 어댑터 모델 불러오기

`PeftModel.from_pretrained()` 메서드를 사용하여 Hugging Face Hub에 업로드된 어댑터 모델을 불러왔습니다. 기본 모델과 어댑터 모델을 결합하여 검증 및 추론에 사용할 준비를 완료했습니다.


In [22]:
# Hugging Face Hub에서 어댑터 로드 (Q4)
# 필요한 클래스 임포트
from peft import PeftModel

# Hugging Face Hub에서 어댑터 로드
adapter_model_id = "hjjummy/meta-llama-1b-lora-adapter_3"  # 업로드된 어댑터 모델 ID
# 학습된 LoRA 어댑터 모델 로드
trained_model = PeftModel.from_pretrained(
    base_model,
    adapter_model_id,  # 업로드된 어댑터 모델 ID
    device_map="auto",
)
print(f"어댑터 모델이 성공적으로 로드되었습니다: {adapter_model_id}")


adapter_config.json:   0%|          | 0.00/734 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/22.6M [00:00<?, ?B/s]

어댑터 모델이 성공적으로 로드되었습니다: hjjummy/meta-llama-1b-lora-adapter_2


In [23]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.6 MB/s eta 0:00:00


In [24]:
# 테스트
preprocessor = function_prepare_sample_text(tokenizer, for_train=False)

In [25]:
preprocessor({'input':'아이스아메리카노 그랑데 한잔 주세요'})

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 07 Dec 2024\n\n이 모델은 주문 문장을 분석하는 역할을 합니다.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n너는 사용자가 입력한 주문 문장을 분석하는 에이전트이다. 주문으로부터 이를 구성하는 음식명, 옵션명, 수량을 차례대로 추출해야 한다.\n### 주문 문장: 아이스아메리카노 그랑데 한잔 주세요<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n'

In [26]:
wrapper_generate(tokenizer=tokenizer, model=trained_model, input_prompt=preprocessor({'input':'아이스아메리카노 그랑데 한잔 주세요. 그리고 베이글 두개요.'}), do_stream=True)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.




- 분석 결과 0: 음식명:아이스아메리카노,옵션:그랑데,수량:한잔
- 분석 결과 1: 음식명:베이글,수량:두개.


'- 분석 결과 0: 음식명:아이스아메리카노,옵션:그랑데,수량:한잔\n- 분석 결과 1: 음식명:베이글,수량:두개.'

## Q5: 검증 세트의 출력에 대해 BLEU 점수 측정

`evaluate` 라이브러리의 `sacrebleu` 메트릭을 사용하여 모델이 생성한 예측 값과 참조 값을 비교해 BLEU 점수를 계산했습니다. 점수는 77.14로, 모델이 참조 데이터와 매우 유사한 텍스트를 생성했음을 보여줍니다.


In [28]:
import evaluate

# sacrebleu 메트릭 로드
bleu_metric = evaluate.load("sacrebleu")

# 예측 및 참조 데이터 생성
predictions = [
    wrapper_generate(tokenizer, trained_model, preprocessor({'input': sample['input']}), do_stream=False)
    for sample in val_dataset
]
references = [[sample["output"]] for sample in val_dataset]  # references는 리스트의 리스트여야 함

# BLEU 점수 계산
results = bleu_metric.compute(predictions=predictions, references=references)

# 결과 출력
print("BLEU Score:", results["score"])
print("Counts:", results["counts"])
print("Totals:", results["totals"])
print("Precisions:", results["precisions"])
print("BP (Brevity Penalty):", results["bp"])
print("System Length:", results["sys_len"])
print("Reference Length:", results["ref_len"])

# 결과를 DataFrame으로 정리
import pandas as pd
df_results = pd.DataFrame.from_dict(results, orient="index", columns=["Value"])
print(df_results)


BLEU Score: 77.13846405009387
Counts: [5399, 5078, 4758, 4444]
Totals: [6665, 6465, 6265, 6065]
Precisions: [81.00525131282821, 78.54601701469451, 75.94573024740623, 73.27287716405606]
BP (Brevity Penalty): 1.0
System Length: 6665
Reference Length: 5567
                                                        Value
score                                               77.138464
counts                               [5399, 5078, 4758, 4444]
totals                               [6665, 6465, 6265, 6065]
precisions  [81.00525131282821, 78.54601701469451, 75.9457...
bp                                                        1.0
sys_len                                                  6665
ref_len                                                  5567


In [29]:
for idx, (prediction, reference) in enumerate(zip(predictions[:5], references[:5])):
    print(f"Prediction {idx + 1}: {prediction}")
    print(f"Reference {idx + 1}: {reference[0]}")
    print("-" * 50)


Prediction 1: - 분석 결과 0: 음식명:꽃살버섯샐러드, 수량:한그릇
- 분석 결과 1: 음식명:스테이크올리오스파게티, 옵션:세트, 수량:세트
- 분석 결과 2: 음식명:꽃살버섯샐러드, 수량:한그릇
- 분석 결과 3: 음식명:스테이크올리오스파게티, 옵션:세트, 수량:세트
Reference 1: - 분석 결과 0: 음식명:꽃살버섯샐러드, 수량:한그릇 
- 분석 결과 1: 음식명:스테이크올리오스파게티
--------------------------------------------------
Prediction 2: - 분석 결과 0: 음식명:교촌통통치킨카츠, 수량:한 판
- 분석 결과 1: 음식명:행복지수커피, 수량:한잔
- 분석 결과 2: 음식명:헤즐넛, 옵션:핫, 수량:하나.
Reference 2: - 분석 결과 0: 음식명:교촌통통치킨카츠, 수량: 한 판
- 분석 결과 1: 음식명:행복지수커피, 수량: 한잔
- 분석 결과 2: 음식명:헤즐넛, 옵션: 핫, 수량: 하나
--------------------------------------------------
Prediction 3: - 분석 결과 0: 음식명:에스프레소도피오,옵션:핫,수량:한잔
- 분석 결과 1: 음식명:허니티,옵션:진하게,수량:한 잔.
Reference 3: - 분석 결과 0: 음식명:에스프레소도피오,옵션:핫,수량:한잔
- 분석 결과 1: 음식명:허니티,옵션:진하게,수량:한 잔
--------------------------------------------------
Prediction 4: - 분석 결과 0: 음식명:밀크티,수량:두 캔
- 분석 결과 1: 음식명:도토리묵밥,수량:한 그릇
- 분석 결과 2: 음식명:동치미,수량:한병.
Reference 4: - 분석 결과 0: 음식명:밀크티,수량:두 캔
- 분석 결과 1: 음식명:도토리묵밥, 수량: 한 그릇
- 분석 결과 2: 음식명:동치미, 수량: 한병
-------------------------------------------

In [30]:
#load_metric가 최신 버전의 datasets 라이브러리에서 제거되었기 때문에 발생하는 오류입니다. 이 문제를 해결하기 위해 evaluate 라이브러리를 사용하는 방식을 적용해야 합니다.

# # BLEU 점수 계산
# from datasets import load_metric

# # sacrebleu 메트릭 로드
# bleu_metric = load_metric("sacrebleu")

# # 예측 및 참조 데이터 생성
# predictions = [
#     wrapper_generate(tokenizer, trained_model, preprocessor({'input': sample['input']}), do_stream=False)
#     for sample in val_dataset
# ]
# references = [[sample["output"]] for sample in val_dataset]  # references는 리스트의 리스트여야 함

# # 메트릭 계산
# bleu_metric.add_batch(predictions=predictions, references=references)
# results = bleu_metric.compute(smooth_method="floor", smooth_value=0.0)

# # 결과 출력
# print("BLEU Score:", results["score"])
# print("Counts:", results["counts"])
# print("Totals:", results["totals"])
# print("Precisions:", results["precisions"])
# print("BP (Brevity Penalty):", results["bp"])
# print("System Length:", results["sys_len"])
# print("Reference Length:", results["ref_len"])

# # 결과를 보기 쉽게 정리
# import pandas as pd
# df_results = pd.DataFrame.from_dict(results, orient="index", columns=["Value"])
# print(df_results)

